In [24]:
import pandas as pd

df = pd.read_csv("../../Data/m4_imputed.csv")


In [25]:
median_wl_time = df['wl_time'].median()
print(f"Median waiting list time: {median_wl_time}")

df['wl_time'] = df['wl_time'].apply(lambda x: 0 if x < median_wl_time else 1)
print(df['wl_time'].value_counts(normalize=True))

Median waiting list time: 78.0
1    0.501886
0    0.498114
Name: wl_time, dtype: float64


In [26]:
import keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataClassifier
from sklearn.metrics import f1_score

X = df.drop(columns=['wl_time'])
y = df['wl_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = StructuredDataClassifier(max_trials=25,
                               project_name="m4_pre_feat",
                               metrics=["accuracy"],
                               overwrite=False)

clf.fit(X_train, y_train, verbose=1)
print(clf.evaluate(X_test, y_test))
f1_s = f1_score(y_test, clf.predict(X_test))
print(f"F1 score: {f1_s}")
print("F1 Macro:", f1_score(y_test, clf.predict(X_test), average='macro'))


Trial 25 Complete [00h 47m 37s]
val_accuracy: 0.6580097079277039

Best val_accuracy So Far: 0.7035598754882812
Total elapsed time: 02h 40m 01s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/28
1936/1936 [==============================] - 4s 2ms/step - loss: 0.6289 - accuracy: 0.6523
Epoch 2/28
1936/1936 [==============================] - 3s 2ms/step - loss: 0.6070 - accuracy: 0.6743
Epoch 3/28
1936/1936 [==============================] - 915s 473ms/step - loss: 0.6022 - accuracy: 0.6800
Epoch 4/28
1936/1936 [==============================] - 3s 2ms/step - loss: 0.5984 - accuracy: 0.6830
Epoch 5/28
1936/1936 [==============================] - 3s 2ms/step - loss: 0.5958 - accuracy: 0.6843
Epoch 6/28
1936/1936 [==============================] - 3s 2ms/step - loss: 0.5938 - accuracy: 0.6861
Epoch 7/28
1936/1936 [==============================] - 921s 476ms/step - loss: 0.5922 - accuracy: 0.6874
Epoch 8/28
1936/1936 [==============================] - 3s 2ms/step - loss: 0.5908 - accuracy: 0.6886
Epoch 9/28
1936/1936 [==============================] - 981s 507ms/step - loss: 0.5895 - accuracy: 0.6894
Epoch 10/28
1936/1936 [==============================] - 3s 2ms/step -

INFO:tensorflow:Assets written to: ./m4_pre_feat/best_model/assets


INFO:tensorflow:Assets written to: ./m4_pre_feat/best_model/assets


484/484 [==============================] - 1s 1ms/step - loss: 0.5956 - accuracy: 0.6884
[0.5956223011016846, 0.6884123682975769]
484/484 [==============================] - 1s 1ms/step
F1 score: 0.6759806555615261
484/484 [==============================] - 1s 1ms/step
F1 Macro: 0.6879530050636693
